# 2.A. What to do with a `DynamicalSystem`? Orbit Diagram
An "orbit diagram" is simply a plot that shows the long term behavior of a discrete system when a parameter is varied. 

How does one compute it?
1. Evolves the system for a transient amount of time.
2. Evolves & saves the output of the system for a chosen amount of time.
3. Changes/increments a parameter of the equations of motion.
4. Repeat steps 1-3 for all given parameter values.

This is exactly what the function `orbitdiagram` does!


---

Let's make the super-ultra-famous orbit diagram of the logistic map:

$$x_{n+1} = rx_n(1-x_n)$$

In [ ]:
using DynamicalSystems, Plots

In [ ]:
logimap = Systems.logistic() # Systems module contains pre-defined well-known systems

---

The call signature of `orbitdiagram` is:

```julia
orbitdiagram(discrete_system, i, p_index, pvalues; n, Ttr, ...)
```
* `i` is the index of the variable we want to save (for which variable to create the orbit diagram).
* `p_index` is the index of the parameter of `discrete_system` that we want to change.
* `pvalues` is the collection of the values that the changing parameter would take on.
* Keywords `Ttr` and `n` denote for how much transient time to evolve the system (that is, how many time steps to throw away from the beginning), and how many steps to save.


In [ ]:
i = 1
n = 2000 # how many values to save
Ttr = 2000 # transient iterations
p_index = 1
pvalues = 2:0.001:4  # parameter values
output = orbitdiagram(logimap, i, p_index, pvalues; n = n, Ttr = Ttr)
typeof(output)

* The output is a vector of vectors. Each inner vector has length `n` and contains the values of the `i`-th variable at the given parameter value.

---

Let's plot this!

In [ ]:
function plot_od(r1, r2, n = 1000, Ttr = 1000)
    params = range(r1, stop = r2, length = 1001)

    res = orbitdiagram(logimap, 1, 1, params; n = n, Ttr = Ttr)
    L = length(params)

    x = Matrix{Float64}(undef, n, L)
    y = Matrix{Float64}(undef, n, L)
    for j in 1:L
        x[:,j] .= params[j]
        y[:,j] .= res[j]
    end
    scatter(x, y, 
        markersize=0.1, markeralpha = 0.3, markercolor="black",
        leg=false, title="Bifurcation graph", 
        html_output_format=:png, size=(2000,1000))
end

In [ ]:
plot_od(2, 4.0)

In [ ]:
plot_od(3.5, 3.6)

---

* `orbitdiagram` works with *any* discrete system! Check out the [documentation page](https://juliadynamics.github.io/DynamicalSystems.jl/latest/chaos/orbitdiagram/) for more!

# 2.B. Poincaré Surface of Section
This is a technique to reduce a continuous system into a discrete map with 1 fewer dimension.
The wikipedia entry on [Poincaré map](https://en.wikipedia.org/wiki/Poincar%C3%A9_map) has a lot of useful info, but the technique itself is very simple:

1. Define a hyperplane in the phase-space of the system. 
2. Evolve the continuous system for long times. Each time the trajectory crosses this plane, record the state of the system.
3. Only crossings with a specific direction (either positive or negative) are allowed.

And that's it! The recorded crossings are the Poincaré Surface of Section!

## Defining a hyperplane
Let's say that our phase-space is $D$ dimensional. If the state of the system is $\mathbf{u} = (u_1, \ldots, u_D)$ then the equation for a hyperplane is 

$$
a_1u_1 + \dots + a_Du_D = \mathbf{a}\cdot\mathbf{u}=b 
$$
where $\mathbf{a}, b$ are the parameters that define the hyperplane.

---

Here is the call signature for a function that does this:

```julia
poincaresos(continuous_system, plane, tfinal = 100.0; kwargs...)
```
In code, `plane` can be either:

* A `Tuple{Int, <: Number}`, like `(j, r)` : the hyperplane is defined as when the `j`-th variable of the system crosses the value `r`.
* An `AbstractVector` of length `D+1`. The first `D` elements of the vector correspond to $\mathbf{a}$ while the last element is $b$. The hyperplane is defined with its formal equation.

---

As an example, let's see a section of the Lorenz system:
$$
\begin{aligned}
\dot{X} &= \sigma(Y-X) \\
\dot{Y} &= -XZ + \rho X -Y \\
\dot{Z} &= XY - \beta Z
\end{aligned}
$$


In [ ]:
lor = Systems.lorenz()

In [ ]:
tr = trajectory(lor, 100.0, dt = 0.005, Ttr = 50.0)
x, y, z = columns(tr)
plot(x,y,z,
        leg=false, title="Lorenz attractor", 
        html_output_format=:png, size=(1000,1000))

**First, let's visualize the Poincaré Surface of Section in 3D**

In [ ]:
c = Vector{String}(undef, length(y))
for i in 1:length(y) # cut points: red
    if -0.1 < y[i] < 0.1
        c[i] = "red"
    elseif y[i] < 0 
        c[i] = "blue" # in front of cut: blue
    else
        c[i] = "green" # behind cut: green
    end
end

In [ ]:
# First let's plot the attractor
plot(x,y,z, color = :black, lw = 1, alpha = 0.25)
scatter!(x, y, z, 
        markersize=2, markeralpha = 0.4, markercolor="black",
        leg=false, title="Lorenz attractor", 
        html_output_format=:png, size=(2000,2000))

In [ ]:
scatter(x, y, z, 
        markersize=3, markeralpha = 0.8, markercolor=c,
        leg=false, title="Lorenz attractor", 
        html_output_format=:png, size=(2000,2000))

In [ ]:
# And then plot the Poincare Surface of Section (PSOS) plane:
function meshgrid(vx, vy)
    m, n = length(vy), length(vx)
    vx = reshape(vx, 1, n); vy = reshape(vy, m, 1)
    (repeat(vx, m, 1), repeat(vy, 1, n))
end
xx = [-20, 20]; zz = [0, 40]; 
X, Z = meshgrid(xx, zz)
Y = zero(X)

plot_surface(X, Y, Z, alpha = 0.25, color = "k");
xlabel("X"); ylabel("Y"); zlabel("Z");

**Now let's use the `poincaresos` function**

In [ ]:
plane = (2, 0.0) # when 2nd variable crosses 0.0

In [ ]:
psos_chaotic = poincaresos(lor, plane, 2000.0, Ttr = 100.0)

In [ ]:
scatter(psos_chaotic[:, 1], psos_chaotic[:, 3],
        markersize=0.15, markeralpha = 0.15, markercolor=:black,
        leg=false, title="Lorenz attractor Poincare section", 
        html_output_format=:png, size=(500,500))

* We see that the surface of section is some kind of 1-dimensional object. 
* This is expected, because as we will show in the tutorial "Entropies & Dimensions" the Lorenz system (at least for the default parameters) lives in an almost 2-dimensional attractor.

* This means that when you take a cut through this object, the result should be 1-dimensional!

Let's now compute the PSOS for a parameter value where the Lorenz system is stable instead of chaotic:

In [ ]:
set_parameter!(lor, 2, 69.75)

In [ ]:
tr = trajectory(lor, 100.0, dt = 0.01, Ttr = 500.0)
x, y, z = columns(tr)
scatter(x, y, z, 
        markersize=1, markeralpha = 0.8,
        leg=false, title="Lorenz attractor")

In [ ]:
psos_regular = poincaresos(lor, (2, 0.0), 2000.0, Ttr = 1000.0)
summary(psos_regular)

In [ ]:
scatter(psos_regular[:, 1], psos_regular[:, 3],
        markersize=0.15, markeralpha = 0.15, markercolor=:black,
        leg=false, title="Non-chaotic Lorenz attractor Poincare section", 
        html_output_format=:png, size=(500,500))

And here are the two different PSOS plots side by side:

In [ ]:
p1 = scatter(psos_chaotic[:, 1], psos_chaotic[:, 3],
        markersize=0.15, markeralpha = 0.15, markercolor=:black,
        leg=false, title="chaotic", 
        html_output_format=:png)

p2 = scatter(psos_regular[:, 1], psos_regular[:, 3],
        markersize=0.15, markeralpha = 0.15, markercolor=:black,
        leg=false, title="nonchaotic", 
        html_output_format=:png)

plot(p1,p2,layout=(1,2),legend=false, size=(1000,500))

# 2.C. Lyapunov exponents

## Definition
Lyapunov exponents measure the exponential separation rate of trajectories that are (initially) close. 

Consider the following picture, where two nearby trajectories are evolved in time:
 

<img src="lyapunov.png" alt="Sketch of the Lyapunov exponent" style="width: 500px;"/>


* $\lambda$ denotes the "maximum Lyapunov exponent".
* A $D$-dimensional system has $D$ exponents.
* In general, a trajectory is called "chaotic" if
    1. it follows nonlinear dynamics
    2. it is *bounded* (does not escape to infinity)
    2. it has at least one positive Lyapunov exponent

*(please be aware that the above is an over-simplification! See the textbooks cited in our documentation for more)*

---

## Demonstration

Before computing Lyapunov exponents, we'll demonstrate the concept of exponential separation using the Henon map that we used before

$$
\begin{aligned}
x_{n+1} &= 1 - ax_n^2 + y_n \\
y_{n+1} &= bx_n
\end{aligned}
$$

In [ ]:
henon = Systems.henon()

First we'll generate a trajectory for the towel map, `tr1`, from the default initial condition,

In [ ]:
tr1 = trajectory(henon, 100)
summary(tr1)

and then we will generate a second trajectory, `tr2`, with a starting point slightly shifted from the initial condition of `tr1`.

In [ ]:
u2 = get_state(henon) + (1e-9 * ones(dimension(henon)))
tr2 = trajectory(henon, 100, u2)
summary(tr2)

In [ ]:
using LinearAlgebra: norm

# Plot the x-coordinate of the two trajectories:
p1 = plot(tr1[:, 1], alpha = 0.5, title="Diverging trajectories")
plot!(tr2[:, 1], alpha = 0.5)

# Plot their distance in a semilog plot:
d = [norm(tr1[i] - tr2[i]) for i in 1:length(tr2)]
p2 = plot(d, yaxis=:log, title="Their distance")

plot(p1,p2,layout=(2,1),legend=false)

### Computing the Lyapunov Exponents

`lyapunov` is a function that calculates the maximum Lyapunov exponent for a DynamicalSystem (for a given starting point).

In [ ]:
λ = lyapunov(henon, 5000) # second argument is time to evolve

This number is _approximately_ the slope of the distance increase!



In [ ]:
using LinearAlgebra: norm

p1 = plot(tr1[:, 1], alpha = 0.5, title="Diverging trajectories")
plot!(tr2[:, 1], alpha = 0.5)

d = [norm(tr1[i] - tr2[i]) for i in 1:length(tr2)]
p2 = plot(d, yaxis=:log, title="Their distance")
plot!(collect(0:50), d[1] .* exp.(collect(0:50) .* λ))

plot(p1,p2,layout=(2,1),legend=false)

If you want to get more than one Lyapunov exponents of a system, use `lyapunovs`

In [ ]:
lyapunovs(henon, 2000)

## Continuous systems

* All functions that accept a `DynamicalSystem` work with *any* instance of `DynamicalSystem`, regardless of whether it is continuous, discrete, in-place, out-of-place, with Jacobian or whatever.
* `lyapunov` and `lyapunovs` both accept a `DynamicalSystem`.

This means that they will "just work" if we use the Lorenz system, `lor`.


In [ ]:
lor = Systems.lorenz()
lyapunov(lor, 2000.0)

In [ ]:
lyapunovs(lor, 2000)

Remember from the Poincare section that for some parameter values the Lorenz system was periodic, for others it was not.

In [ ]:
ρs = (69.75, 28.0)
p = []
c = ["red", "blue"]
for (i, ρ) in enumerate(ρs)
    set_parameter!(lor, 2, ρ)
    psos = poincaresos(lor, (2, 0.0), 2000.0, Ttr = 2000.0)
    pi = scatter(psos[:, 1], psos[:, 3],
        markersize=0.15, markeralpha = 0.15, markercolor=c[i], 
        title=string("\\rho", " = $ρ, ", (i != 1 ? "not periodic" : "periodic")))
    push!(p, pi)
end

plot(p[1],p[2],layout=(1,2),legend=false, size=(1000,500))

In [ ]:
plot(1,1, title="\\rho")

Seems like the exponent in the first case λ should be equal to zero, and in the second λ should be positive.

In [ ]:
ρs = (69.75, 28.0)

for (i, ρ) in enumerate(ρs)
    set_parameter!(lor, 2, ρ)
    λ = lyapunov(lor, 2000.0; Ttr = 2000.0)
    println("For ρ = $ρ, λ = $λ")
end

One has to be **very careful** when using functions like `lyapunovs`. They are approximative methods! Naively doing short computations or not using large transient times can lead to wrong results!

In [ ]:
for (i, ρ) in enumerate(ρs)
    set_parameter!(lor, 2, ρ)
    λ = lyapunov(lor, 200.0) # smaller integration time, no transient time
    println("For ρ = $ρ, λ = $λ")
end

## Benchmarks

The Lyapunov exponent computations are quite fast! To benchmark them we can use the `BenchmarkTools` package.

In [ ]:
using BenchmarkTools, OrdinaryDiffEq

In [ ]:
diffeq = (reltol = 1e-6, abstol = 1e-6) 

In [ ]:
@btime lyapunovs($lor, 2000; Ttr = 200, $diffeq...); # use default solver SimpleATsit5()

For performance it is always imprortant to choose the correct solver! For example, even though `Vern9()` is a higher order solver than the default, it is faster for computing the exponent:

In [ ]:
@btime lyapunovs($lor, 2000; Ttr = 200, $diffeq..., alg = Vern9());

This happens because `Vern9` can take larger steps. For more information visit the official documentation of **DynamicalSystems.jl**.

---

Discrete systems are also super faster!

In [ ]:
tow = Systems.towel() # 3D discrete chaotic system

In [ ]:
@btime lyapunovs($tow, 2000; Ttr = 200);

In [ ]:
@btime lyapunov($tow, 2000; Ttr = 200);

# 2.D. How **DynamicalSystems.jl** is a library

What if the command `lyapunov(ds, 2000)` is too magical for you? Given that you have a basic idea of what a Lyapunov exponent is, you still don't know _how_ the function `lyapunov` actually computes it.

And knowing _exactly_ how a function computes stuff is really important for science!

For this, we turn to its documentation string:

In [ ]:
?lyapunov

---

**All** documentation strings of functions exported by **DynamicalSystems.jl** have this structure:

1. Call signature
2. Couple of sentences explanation of what the function does
2. Keyword argument listing
3. Scientific description of the algorithm used
4. References

This way not only you get the basic idea of how the function works, but if this is not enough for you and want to learn more, you can look at the references!

# 2.E. More library: the GALI algorithm

For this part we will simply visit the documentation page of **DynamicalSystems.jl** and use the search function.

https://juliadynamics.github.io/DynamicalSystems.jl/latest/